In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import pickle

from tqdm import tqdm

from collections import Counter

from linearization.analyzer import SAELinearizer
from linearization.visualization import visualize_topk

In [3]:
# Hyperparameters
# SAE_NUM = "0_8192"
# SAE_NUM = "1_32768"
# SAE_NUM = "2_32768"


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

N_TOP_TOKENS = 15

NORMALIZE_SAE_VECS = False
# NORMALIZE_SAE_VECS = True

In [33]:
path = "../data/pickled_linearizers/gpt2_small_resid_mid.pkl"

if os.path.exists(path):
    lin = pickle.load(open(path, "rb"))
else:
    lin = SAELinearizer(
        model_name="gpt2-small",
        sae_names=[
            "../scripts/checkpoints/64x_resid_mid/layer0/final_sparse_autoencoder_gpt2-small_blocks.0.hook_resid_mid_49152.pt"
        ],
        layers=[0],
        dataset_name="NeelNanda/pile-10k",
        act_name="resid_mid",
        num_batches=10,
        use_gpt=True,
        # dict_mult=64,
        # hook_point="resid_mid",
        # run_analysis=False,
    )
    with open(path, "wb") as f:
        pickle.dump(lin, f)

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda
Changing model dtype to torch.float16
Model device: cuda:0
Tokens shape: torch.Size([136625, 128]), dtype: torch.int64, device: cuda:0
Loading GPT2-small layer ../scripts/checkpoints/64x_resid_mid/layer0/final_sparse_autoencoder_gpt2-small_blocks.0.hook_resid_mid_49152.pt from disk
Encoder device: cuda:0
dict_keys(['../scripts/checkpoints/64x_resid_mid/layer0/final_sparse_autoencoder_gpt2-small_blocks.0.hook_resid_mid_49152.pt'])


100%|██████████| 10/10 [00:00<00:00, 13.72it/s]


Num dead 0.6428426504135132


100%|██████████| 10/10 [02:54<00:00, 17.46s/it]


PicklingError: Can't pickle <class 'linearization.sae_tutorial.AutoEncoder'>: it's not the same object as linearization.sae_tutorial.AutoEncoder

In [24]:
import torch

sae = torch.load(
    "/home/phil/mlp_linearization/scripts/checkpoints/64x_resid_mid/layer0/final_sparse_autoencoder_gpt2-small_blocks.0.hook_resid_mid_49152.pt"
)
sae.keys()

dict_keys(['cfg', 'state_dict'])

In [28]:
sae["cfg"]

LanguageModelSAERunnerConfig(model_name='gpt2-small', hook_point='blocks.0.hook_resid_mid', hook_point_layer=0, hook_point_head_index=None, dataset_path='Skylion007/openwebtext', is_dataset_tokenized=False, context_size=128, use_cached_activations=False, cached_activations_path='activations/Skylion007_openwebtext/gpt2-small/blocks.0.hook_resid_mid', d_in=768, n_batches_in_buffer=128, total_training_tokens=300000000, store_batch_size=32, device=device(type='cuda'), seed=42, dtype=torch.float32, b_dec_init_method='geometric_median', expansion_factor=64, from_pretrained_path=None, l1_coefficient=8e-05, lr=0.0004, lr_scheduler_name='constantwithwarmup', lr_warm_up_steps=5000, train_batch_size=4096, use_ghost_grads=True, feature_sampling_window=1000, feature_sampling_method=None, resample_batches=32, feature_reinit_scale=0.2, dead_feature_window=5000, dead_feature_estimation_method='no_fire', dead_feature_threshold=1e-06, log_to_wandb=False, wandb_project='mats_sae_training_gpt2', wandb_ent